In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mobile-price-classification/train.csv
/kaggle/input/mobile-price-classification/test.csv


In [25]:
train = pd.read_csv('/kaggle/input/mobile-price-classification/train.csv')

In [37]:
train.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [38]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [27]:
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('price_range', axis=1),
                                                   train['price_range'],
                                                   test_size=0.5, random_state=42)

In [5]:
cont_vars = ['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep',
            'mobile_wt', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w',
            'talk_time']
cat_vars = ['blue', 'dual_sim', 'four_g', 'n_cores', 'three_g', 'touch_screen', 'wifi']
# to predict - the categorical variable, price_range.

In [6]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [19]:
column_transformer_pipeline = ColumnTransformer([
    ('numeric', StandardScaler(), cont_vars),
    ('categorical', OneHotEncoder(), cat_vars)
])
logistic_regression = LogisticRegression(multi_class = 'multinomial',
                                         solver='saga',
                                         penalty='l2',
                                         max_iter = 1000,
                                         n_jobs=-1, random_state=42)
#pipeline = Pipeline([
#    ('preprocessing', column_transformer_pipeline),
#    ('logistic_regression', logistic_regression)
#])
#predictor = pipeline.fit(X=train.drop('price_range', axis=1), y=train['price_range'])

In [39]:
preprocessor = column_transformer_pipeline.fit(X_train)
X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

In [40]:
scores = cross_val_score(logistic_regression, X_train, y_train, cv=10,
               n_jobs=-1,scoring='accuracy')
f1_micro = cross_val_score(logistic_regression, X_train, y_train, cv=10,
               n_jobs=-1,scoring='f1_micro')
roc_auc_ovr = cross_val_score(logistic_regression, X_train, y_train, cv=10,
               n_jobs=-1,scoring='roc_auc_ovr')

In [41]:
scores.mean().round(3), scores.std().round(3)

(0.938, 0.019)

In [42]:
f1_macro.mean().round(3), f1_macro.std().round(3)

(0.972, 0.011)

In [43]:
roc_auc_ovr.mean().round(3), roc_auc_ovr.std().round(3)

(0.995, 0.003)

In [44]:
y_pred = logistic_regression.fit(X_train, y_train).predict(X_test)

In [48]:
from sklearn.metrics import classification_report

In [52]:
print(classification_report(y_test, y_pred, target_names = ['low_cost','medium_cost',
                                                           'high_cost', 'very_high_cost']))

                precision    recall  f1-score   support

      low_cost       0.98      0.97      0.97       257
   medium_cost       0.92      0.94      0.93       248
     high_cost       0.94      0.92      0.93       252
very_high_cost       0.97      0.98      0.98       243

      accuracy                           0.95      1000
     macro avg       0.95      0.95      0.95      1000
  weighted avg       0.95      0.95      0.95      1000

